# --- `Day 23`: Title ---

In [1]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [ ]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [ ]:
final_input = parse_input(aocd.get_data(day=23, year=2021))
print(final_input[:5])

In [11]:
test_input = parse_input('''\
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
''')

print(test_input)

['#############', '#...........#', '###B#C#B#D###', '  #A#D#C#A#', '  #########']


### Helpers

In [2]:
def printBoard(board):
    print("")
    hallway, cave = board
    print("#############")
    print(f"#{''.join(hallway)}#")
    print(f"###{cave[0][0]}#{cave[1][0]}#{cave[2][0]}#{cave[3][0]}###")
    print(f"  #{cave[0][1]}#{cave[1][1]}#{cave[2][1]}#{cave[3][1]}#  ")
    print("  #########")
    
def isCaveOpen(board, piece):
    _, cave = board
    if piece == 'A':
        if cave[0][0] == '.' and cave[0][1] == '.':
            return 1
        elif cave[0][0] == '.' and cave[0][1] == 'A':
            return 0
    elif piece == 'B':
        if cave[1][0] == '.' and cave[1][1] == '.':
            return 1
        elif cave[1][0] == '.' and cave[1][1] == 'B':
            return 0
    elif piece == 'C':
        if cave[2][0] == '.' and cave[2][1] == '.':
            return 1
        elif cave[2][0] == '.' and cave[2][1] == 'C':
            return 0
    elif piece == 'D':
        if cave[3][0] == '.' and cave[3][1] == '.':
            return 1
        elif cave[3][0] == '.' and cave[3][1] == 'D':
            return 0
    return None

def get00Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[1] == '.':
        moves.append((piece, source, 1, 2 + spaces))
        if hallway[0] == '.':
            moves.append((piece, source, 0, 3 + spaces))
    if hallway[3] == '.':
        moves.append((piece, source, 3, 2 + spaces))
        if piece == 'B' and caveOpen != None:
            moves.append((piece, source, 13 + caveOpen, 4 + spaces + caveOpen))
        if hallway[5] == '.':
            moves.append((piece, source, 5, 4 + spaces))
            if piece == 'C' and caveOpen != None:
                moves.append((piece, source, 15 + caveOpen, 6 + spaces + caveOpen))
            if hallway[7] == '.':
                moves.append((piece, source, 7, 6 + spaces))
                if piece == 'D' and caveOpen != None:
                    moves.append((piece, source, 17 + caveOpen, 8 + spaces + caveOpen))
                if hallway[9] == '.':
                    moves.append((piece, source, 9, 8 + spaces))
                    if hallway[10] == '.':
                        moves.append((piece, source, 10, 9 + spaces))
    return moves

def get10Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[3] == '.':
        moves.append((piece, source, 3, 2 + spaces))
        if piece == 'A' and caveOpen != None:
            moves.append((piece, source, 11 + caveOpen, 4 + spaces + caveOpen))
        if hallway[1] == '.':
            moves.append((piece, source, 1, 4 + spaces))
            if hallway[0] == '.':
                moves.append((piece, source, 0, 5 + spaces))
    if hallway[5] == '.':
        moves.append((piece, source, 5, 2 + spaces))
        if piece == 'C' and caveOpen != None:
            moves.append((piece, source, 15 + caveOpen, 4 + spaces + caveOpen))
        if hallway[7] == '.':
            moves.append((piece, source, 7, 4 + spaces))
            if piece == 'D' and caveOpen != None:
                moves.append((piece, source, 17 + caveOpen, 6 + spaces + caveOpen))
            if hallway[9] == '.':
                moves.append((piece, source, 9, 6 + spaces))
                if hallway[10] == '.':
                    moves.append((piece, source, 10, 7 + spaces))
    return moves

def get20Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[5] == '.':
        moves.append((piece, source, 5, 2 + spaces))
        if piece == 'B' and caveOpen != None:
            moves.append((piece, source, 13 + caveOpen, 4 + spaces + caveOpen))
        if hallway[3] == '.':
            moves.append((piece, source, 3, 4 + spaces))
            if piece == 'A' and caveOpen != None:
                moves.append((piece, source, 11 + caveOpen, 6 + spaces + caveOpen))
            if hallway[1] == '.':
                moves.append((piece, source, 1, 6 + spaces))
                if hallway[0] == '.':
                    moves.append((piece, source, 0, 7 + spaces))
    if hallway[7] == '.':
        moves.append((piece, source, 7, 2 + spaces))
        if piece == 'D' and caveOpen != None:
            moves.append((piece, source, 17 + caveOpen, 4 + spaces + caveOpen))
        if hallway[9] == '.':
            moves.append((piece, source, 9, 4 + spaces))
            if hallway[10] == '.':
                moves.append((piece, source, 10, 5 + spaces))
    return moves

def get30Moves(board, source, piece, spaces):
    hallway, cave = board
    moves = []
    caveOpen = isCaveOpen(board, piece)
    if hallway[7] == '.':
        moves.append((piece, source, 7, 2 + spaces))
        if piece == 'C' and caveOpen != None:
            moves.append((piece, source, 15 + caveOpen, 4 + spaces + caveOpen))  
        if hallway[5] == '.':
            moves.append((piece, source, 5, 4 + spaces))
            if piece == 'B' and caveOpen != None:
                moves.append((piece, source, 13 + caveOpen, 6 + spaces + caveOpen))
            if hallway[3] == '.':
                moves.append((piece, source, 3, 6 + spaces))
                if piece == 'A' and caveOpen != None:
                    moves.append((piece, source, 11 + caveOpen, 8 + spaces + caveOpen))
                if hallway[1] == '.':
                    moves.append((piece, source, 1, 8 + spaces))
                    if hallway[0] == '.':
                        moves.append((piece, source, 0, 9 + spaces))
    
    if hallway[9] == '.':
        moves.append((piece, source, 9, 2 + spaces))
        if hallway[10] == '.':
            moves.append((piece, source, 10, 3 + spaces))
    return moves

def getMoves(board):
    hallway, cave = board
    moves = []
    # from hallway
    places = {'A':11, 'B':13, 'C':15, 'D':17}
    for i,n in enumerate(hallway):
        if n != '.':
            caveOpen = isCaveOpen(board, n)
            if caveOpen != None:
                times = None
                if n == 'A':
                    if i == 0 and hallway[1] == '.':
                        times = 3
                    elif i == 1 or i == 3:
                        times = 2
                    elif i == 5 and hallway[3] == '.':
                        times = 4
                    elif i == 7 and hallway[3] == '.' and hallway[5] == '.':
                        times = 6
                    elif i == 9 and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.':
                        times = 8
                    elif i == 10 and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.' and hallway[9] == '.':
                        times = 9
                elif n == 'B':
                    if i == 0 and hallway[1] == '.' and hallway[3] == '.':
                        times = 5
                    elif i == 1 and hallway[3] == '.':
                        times = 4
                    elif i == 3 or i == 5:
                        times = 2
                    elif i == 7 and hallway[5] == '.':
                        times = 4
                    elif i == 9 and hallway[5] == '.' and hallway[7] == '.':
                        times = 6
                    elif i == 10 and hallway[5] == '.' and hallway[7] == '.' and hallway[9] == '.':
                        times = 7
                elif n == 'C':
                    if i == 0 and hallway[1] == '.' and hallway[3] == '.' and hallway[5] == '.':
                        times = 7
                    elif i == 1 and hallway[3] == '.' and hallway[5] == '.':
                        times = 6
                    elif i == 3 and hallway[5] == '.':
                        times = 4
                    elif i == 5 or i == 7:
                        times = 2
                    elif i == 9 and hallway[7] == '.':
                        times = 4
                    elif i == 10 and hallway[7] == '.' and hallway[9] == '.':
                        times = 5
                elif n == 'D':
                    if i == 0 and hallway[1] == '.' and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.':
                        times = 9
                    elif i == 1 and hallway[3] == '.' and hallway[5] == '.' and hallway[7] == '.':
                        times = 8
                    elif i == 3 and hallway[5] == '.' and hallway[7] == '.':
                        times = 6
                    elif i == 5 and hallway[7] == '.':
                        times = 4
                    elif i == 7 or i == 9:
                        times = 2
                    elif i == 10 and hallway[9] == '.':
                        times = 3
                if times != None:
                    moves.append((n, i, places[n] + caveOpen, times + caveOpen))
    
    # from cave
    if cave[0][0] != 'A' or cave[0][1] != 'A':
        if cave[0][0] != '.':
            moves.extend(get00Moves(board, 11, cave[0][0], 0))
        elif cave[0][1] in ['B', 'C', 'D'] and cave[0][0] == '.':
            moves.extend(get00Moves(board, 12, cave[0][1], 1))
    if cave[1][0] != 'B' or cave[1][1] != 'B':
        if cave[1][0] != '.':
            moves.extend(get10Moves(board, 13, cave[1][0], 0))
        elif cave[1][1] in ['A', 'C', 'D'] and cave[1][0] == '.':
            moves.extend(get10Moves(board, 14, cave[1][1], 1))
    if cave[2][0] != 'C' or cave[2][1] != 'C':
        if cave[2][0] != '.':
            moves.extend(get20Moves(board, 15, cave[2][0], 0))
        elif cave[2][1] in ['A', 'B', 'D'] and cave[2][0] == '.':
            moves.extend(get20Moves(board, 16, cave[2][1], 1))
    if cave[3][0] != 'D' or cave[3][1] != 'D':
        if cave[3][0] != '.':
            moves.extend(get30Moves(board, 17, cave[3][0], 0))
        elif cave[3][1] in ['A', 'B', 'C'] and cave[3][0] == '.':
            moves.extend(get30Moves(board, 18, cave[3][1], 1))
    return moves

## Solution 1

In [6]:
def checkWin(board):
    hallway, cave = board
    if cave[0][0] == cave[0][1] == 'A':
        if cave[1][0] == cave[1][1] == 'B':
            if cave[2][0] == cave[2][1] == 'C':
                if cave[3][0] == cave[3][1] == 'D':
                    return True
    return False
    
def updateCave(caves, piece, source, dest):
    newCaves = [[x,y] for x,y in caves]
    if source == 11:
        newCaves[0][0] = '.'
    elif source == 12:
        newCaves[0][1] = '.'
    elif source == 13:
        newCaves[1][0] = '.'
    elif source == 14:
        newCaves[1][1] = '.'
    elif source == 15:
        newCaves[2][0] = '.'
    elif source == 16:
        newCaves[2][1] = '.'
    elif source == 17:
        newCaves[3][0] = '.'
    elif source == 18:
        newCaves[3][1] = '.'
        
    if dest == 11:
        newCaves[0][0] = piece
    elif dest == 12:
        newCaves[0][1] = piece
    elif dest == 13:
        newCaves[1][0] = piece
    elif dest == 14:
        newCaves[1][1] = piece
    elif dest == 15:
        newCaves[2][0] = piece
    elif dest == 16:
        newCaves[2][1] = piece
    elif dest == 17:
        newCaves[3][0] = piece
    elif dest == 18:
        newCaves[3][1] = piece
        
    return tuple([(x,y) for x,y in newCaves])
    
@lru_cache(None)
def move(board):
    hallway, cave = board
    costs = {'A':1, 'B':10, 'C':100, 'D':1000}
    
    if checkWin(board):
        return 0
    moves = getMoves(board)
    if len(moves) == 0:
        return None
    best = None
    for piece, source, dest, spaces in moves:
        newBoard = tuple()
        if dest <= 10:
            newHallway = [x for x in hallway]
            newHallway[dest] = piece
            newHallway = tuple(newHallway)
            newCave = updateCave(cave, piece, source, dest)
            newBoard = (newHallway, newCave)
        elif source <= 10:
            newHallway = [x for x in hallway]
            newHallway[source] = '.'
            newHallway = tuple(newHallway)
            newCave = updateCave(cave, piece, source, dest)
            newBoard = (newHallway, newCave)
        else:
            newCave = updateCave(cave, piece, source, dest)
            newBoard = (hallway, newCave)
        #printBoard(newBoard)
        #print(spaces)
        temp = move(newBoard)
        #temp = None
        if temp != None:
            cost = costs[piece] * spaces
            temp += cost
            if best == None or temp < best:
                best = temp
    if best == None:
        return None
    return best  

def solve_1():
    hallway = ['.'] * 11
    hallway = tuple(hallway)
    #caves = (("B", "A"), ("C", "D"), ("B", "C"), ("D", "A"))
    caves = (("C", "D"), ("A", "C"), ("B", "A"), ("D", "B"))
    board = (hallway, caves)
    printBoard(board)
    cost = move(board)
    return cost
           
solve_1()
#15358
#51436


#############
#...........#
###C#A#B#D###
  #D#C#A#B#  
  #########


15358

In [228]:
f"Solution 1: {solve_1(final_input)}"

NameError: name 'final_input' is not defined

## Solution 2

In [ ]:
def solve_2(input):
    return ""
    
solve_2(test_input)

In [ ]:
f"Solution 2: {solve_2(final_input)}"